In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

In [81]:
AllData = pd.read_csv('./dataFolders/compiledTrackData.csv')
circ_params = pd.read_csv('./dataFolders/circle_parameters.csv')
outfig = './dataFolders/Figures/compareTrueAndGaussian/'

In [3]:
AllData.head()

,Unnamed: 0,name,trial,xpts,ypts,radius,theta
0,0,c-1_m13,0,NaN,NaN,NaN,NaN
1,1,c-1_m13,0,5.011057,348.049824,321.678510,160.373100
2,2,c-1_m13,0,20.932432,335.463658,302.524542,161.605573
3,3,c-1_m13,0,40.568796,325.165885,280.664705,162.335453
4,4,c-1_m13,0,50.652334,323.449590,270.539563,162.033785


In [4]:
names = AllData.name.unique()
names

array(['c-1_m13', 'c-2_m5', 'c-3_m10', 'c-10_m11', 'c-1_m2', 'c-1_m6',
       'c-2_m1', 'c-2_m11', 'c-3_m1', 'c-3_m6', 'c-10_m3', 'c-10_m6'],
      dtype=object)

In [44]:
# read in the tracked and data

path = './dataFolders/Final Tracking xypts/AngleAndRadiusIncluded/'
dlc_file = glob.glob(path + '**.csv')

In [76]:
def comparisionFig(manual, dlc, gaussian):
    fig, ax = plt.subplots(figsize = (20,10))

    plt.plot(manual, label = 'true')
    plt.plot(dlc, label = 'dlc')
    # plt.plot(dlc_data.interpol_x, 'o', label = 'interpolated')
    plt.plot(gaussian, linewidth = 3, label = 'gaussian')
    plt.legend()
    return(fig, ax)

In [85]:
for name in names:

    # read the manual data
    circ_x = circ_params.circ_x[circ_params.name == name].values
    circ_y = circ_params.circ_y[circ_params.name == name].values
    circ_r = circ_params.circ_radii[circ_params.name == name].values

    for trial in ['first', 'last']:
        if trial == 'first':
            var = '0'
        else:
            var = 'last'

        x = AllData[(AllData.name == name) & (AllData.trial == var)].xpts.values
        y = AllData[(AllData.name == name) & (AllData.trial == var)].ypts.values
        # correct for inversion

        y = 488 - y

        # center the x and y coordinates
        cent_x = x - circ_x
        cent_y = y - circ_y
        r = np.linalg.norm([cent_x, cent_y], axis = 0)
        r = r/circ_r

        # read the dlc data
        file = [f for f in dlc_file if (name in f) & (trial in f)][0]
        dlc_data = pd.read_csv(file)

        f1, ax1 = comparisionFig(cent_x, dlc_data.cent_x, dlc_data.x_gauss)
        f1.savefig(outfig + name + '_' + trial + '_x.png')
        f2, ax2 = comparisionFig(cent_y, dlc_data.cent_y, dlc_data.y_gauss)
        f2.savefig(outfig + name + '_' + trial + '_y.png')
        f3, ax3 = comparisionFig(r, dlc_data.radius_interpol, dlc_data.r_gauss)
        f3.savefig(outfig + name + '_' + trial + '_radius.png')
        plt.close('all')